In [1]:
import numpy as np
import math
from astropy.convolution import Gaussian2DKernel, Kernel2D, Moffat2DKernel
from astropy.modeling import models
from astropy.constants import h,c,k_B
import matplotlib.pyplot as plt
import scipy

h = h.value
c = c.value
k_B = k_B.value

def _round_up_to_odd_integer(value):
    i = math.ceil(value)
    if i % 2 == 0:
        return i + 1
    else:
        return i
    
class Moffat2DKernel2(Kernel2D):

    _is_bool = False

    def __init__(self, amplitude, x_0, y_0, gamma, alpha, **kwargs):
        self._model = models.Moffat2D(amplitude, x_0, y_0, gamma, alpha)
        #print(self._model.fwhm)
        self._default_size = _round_up_to_odd_integer(4.0 * self._model.fwhm)
        super().__init__(**kwargs)
        #self.normalize()
        self._truncation = None
        
def blackbody(l,T=10000):
    '''
    Inputs
    ------
    l in angstrom
    T in kelvin
    '''
    l = l*1e-10 # converting to meters
    bb = 2*h*c**2/l**5*(1/(np.exp(h*c/(l*k_B*T))-1))*1e-22
    bb /= bb.sum()
    return bb

def moffat(seeing, amplitude = None, center = (0,0),mode=None,factor=None, size=None):
    alpha = 3.5
    gamma = 0.5 * seeing / np.sqrt(2**(1./alpha) - 1.)
    if amplitude == None:
        amplitude = (alpha - 1.0) / (np.pi * gamma * gamma)
    moff = Moffat2DKernel2(amplitude,
                            center[0],
                            center[1],
                            gamma,
                            alpha,
                            x_size=size,
                            mode=mode,
                            factor=factor)
    return moff.array

def gaussian(seeing):
    morph = Gaussian2DKernel(seeing).array
    return morph

def spectral_source(spectra,morph):
    cube = np.outer(spectra.T, morph)
    cube.shape = (len(spectra), morph.shape[0], morph.shape[1])
    return cube



In [2]:
wl = np.arange(3470.0,5542.0,2)

In [3]:
from astropy.io import fits
import numpy as np
import os 
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import matplotlib.pyplot as plt

ecube="/home/majoburo/Work/GWHET-paper/cubes/20191214_0000023_047_error_cube.fits"
ccube="/home/majoburo/Work/GWHET-paper/cubes/20191214_0000023_047cutout.fits"
cube="/home/majoburo/Work/GWHET-paper/cubes/20191214_0000023_047_cube.fits"


def getsex(fname):
    os.chdir("../config")
    os.system("sextractor "+fname)
    sexdata = np.loadtxt('test.cat')
    os.chdir("../junk")
    return sexdata


path,file = os.path.split(cube)

#kn0 = path+"/kn0_"+file
hdu = fits.open(cube)

#chdu = fits.open(ccube)
#chduu = chdu.__deepcopy__()
#chduu[0].data = chdu[0].data#.sum(axis=0)
#chduu.writeto(kn0, overwrite=True)
sexdata = getsex(ccube)

#fig, ax = plt.subplots()
#print(sexdata)
#from matplotlib.patches import Ellipse
#import matplotlib.transforms as transforms
#x,y=sexdata[0,6:8]
#a,b,theta=sexdata[0,11:14]
#print(x,y,a,b,theta)
#ellipse = Ellipse((0,0),
#        width=a * 2,
#        height=b * 2,
#        facecolor='none',
#        edgecolor='red')
#transf = transforms.Affine2D() \
#        .rotate_deg(theta)\
#        .translate(x-1,y-1)
#
#ellipse.set_transform(transf + ax.transData)
##ax.imshow(chduu[0].data)
#ax.add_patch(ellipse)
#x1,y1=[],[]
#noise = np.zeros(chduu[0].data.shape)
#for i in range(3):
#    x1.append(x+i*a*np.cos(theta*np.pi/180)-1)
#    y1.append(y+i*a*np.sin(theta*np.pi/180)-1)
#    noise += moffat(2.2*1/0.75,amplitude=30,center=(x1[i]-63/2+1/2,y1[i]-63/2+1/2),mode='oversample',factor=20,size=63)
#ax.imshow(chduu[0].data + noise)
##ax.imshow(noise)
#
#plt.scatter(x1,y1,color="red")
#plt.scatter(sexdata[:,6]-1,sexdata[:,7]-1,c='orange',marker='+')
#plt.show()
#



In [4]:
def makeknfits(amp,x1,y1,bb,hdu,kn0):
    morph = moffat(2.2*1/0.75,amplitude=amp,center=(x1-63/2+1/2,y1-63/2+1/2),mode='oversample',factor=20,size=63)
    kn = spectral_source(bb,morph)
    hduu = hdu.__deepcopy__()
    hduu[0].header['INSTRUME']="injection"
    hduu[0].data = hduu[0].data + kn
    path,knfile = os.path.split(kn0)
    kn0 = path+"/%s_"%amp+knfile
    hduu.writeto(kn0, overwrite=True)
    hduu[0].data = (hduu[0].data + kn).sum(axis=0)
    fkn0 = path+"/f%s_"%amp+knfile
    hduu.writeto(fkn0, overwrite=True)
    return

def detectablesex(amp,x1,y1,bb,hdu,kn0):
    #Ugly ass function
    morph = moffat(2.2*1/0.75,amplitude=amp,center=(x1-63/2+1/2,y1-63/2+1/2),mode='oversample',factor=20,size=63)
    kn = spectral_source(bb,morph)
    hduu = hdu.__deepcopy__()
    hduu[0].header['INSTRUME']="injection"
    hduu[0].data = hduu[0].data + kn
    path,knfile = os.path.split(kn0)
    kn0=path+"/%s_"%amp+knfile
    hduu.writeto(kn0, overwrite=True)
    hduu[0].data = (hduu[0].data + kn).sum(axis=0)
    fkn0=path+"/f%s_"%amp+knfile
    hduu.writeto(fkn0, overwrite=True)
    #plt.imshow((hduu[0].data))
    points = getsex(fkn0)
    xx,yy,dx2,dy2 = points[:,6:10].T
    xx,yy = points[:,6:8].T
    xx=xx-1
    yy=yy-1
    #plt.scatter(xx,yy)
    #plt.show()
    #ar = np.argmin((xx-x1)**2+(yy-y1)**2)
    #print(xx[ar],x1,yy[ar],y1,dx2[ar],dy2[ar])
    return np.any(np.sqrt((xx-x1)**2+(yy-y1)**2) <= 1)

def detectablescar(amp,x1,y1,bb,hdu,kn0):
    #Ugly ass function
    morph = moffat(2.2*1/0.75,amplitude=amp,center=(x1-63/2+1/2,y1-63/2+1/2),mode='oversample',factor=20,size=63)
    kn = spectral_source(bb,morph)
    hduu = hdu.__deepcopy__()
    hduu[0].header['INSTRUME']="injection"
    hduu[0].data = hduu[0].data + kn
    path,knfile = os.path.split(kn0)
    kn0=path+"/%s_"%amp+knfile
    hduu.writeto(kn0, overwrite=True)
    import spaxlet
    xx,yy = spaxlet.main(kn0)
    #xx,yy,dx2,dy2 = points[:,6:10].T
    #xx,yy = points[:,6:8].T
    #xx=xx-1
    #yy=yy-1
    #plt.scatter(xx,yy)
    #plt.show()
    #ar = np.argmin((xx-x1)**2+(yy-y1)**2)
    #print(xx[ar],x1,yy[ar],y1,dx2[ar],dy2[ar])
    return np.any(np.sqrt((xx-x1)**2+(yy-y1)**2) <= 1)



In [5]:
def kn_in_gal(sexdata, ccube, plot = False):
    #GET GALACTIC SHAPE PARAMS
    cdata = sexdata[np.all((sexdata[:,6:8]<63/2+5)*
                           (sexdata[:,6:8]>63/2-5),axis=1)]
    if len(cdata) > 1:
        cdata = np.argmin(cdata[:,-1])
    if len(cdata) == 0:
        raise("No galaxy within central radius. Is this a transient follow-up?")
    a,b,theta = cdata[0][11:14]
    x,y = cdata[0][6:8]
    
    #Copy the cube
    hdu = fits.open(cube)    
    #place kn in different positions and temperatures
    temps = [4000,7000,11000]
    a_away = range(1,4) #number of semi-major axis away
    th_amp = {}
    for T in temps: #,7000,11000]:
        bb = blackbody(wl, T)
        for i in a_away:
            x1 = x+(i)*a*np.cos(theta*np.pi/180)-1
            y1 = y+(i)*a*np.sin(theta*np.pi/180)-1
            print(x1,y1)
            kn0 = "%s/kn_%d_%d_%s"%(path,i,T,file)
            print(kn0)
            amp = binary_search(x1,y1,bb,hdu,kn0,low=10,high=200)
            th_amp[(T,i)] = amp
            if plot:
                sexdataplot = np.loadtxt('../config/test.cat')
                xp,yp = sexdataplot[:,6:8].T
                #a,b,theta = sexdata[0,11:14]
                #print(os.getcwd())
                fig, ax = plt.subplots()
                #ellipse = Ellipse((0,0),
                #        width = a * 2,
                #        height = b * 2,
                #        facecolor = 'none',
                #        edgecolor = 'red')
                #transf = transforms.Affine2D() \
                #        .rotate_deg(theta)\
                #        .translate(x-1,y-1)

                #ellipse.set_transform(transf + ax.transData)
                ax.imshow(hdu[0].data.sum(axis=0))
                #ax.add_patch(ellipse)
                #ax.imshow((hduu[0].data + kn).sum(axis=0))
                plt.scatter(xp-1,yp-1,color="white")
                plt.show()
    return th_amp

def binary_search(x1,y1,bb,hdu,kn0,low=10,high=1000):
    #def detectable(i):
    #return i > num
    #low = 10
    #high = 1000
    amp = (high + low)/2
    iterations = 0
    num_iterations = int(np.log(high-low)/np.log(2))
    while iterations < num_iterations:
        if detectablescar(amp,x1,y1,bb,hdu,kn0):
            high = amp
        else:
            low = amp
        amp = (high+low)/2
        iterations +=1
    return amp

In [ ]:
th_amp=kn_in_gal(sexdata, ccube, plot = False)


29.285293506811094 29.08165612099596
/home/majoburo/Work/GWHET-paper/cubes/kn_1_4000_20191214_0000023_047_cube.fits
scarlet ran for 84 iterations to logL = 2690979.29586852
scarlet ran for 84 iterations to logL = 2690988.1441587773
scarlet ran for 82 iterations to logL = 2690993.394535131
scarlet ran for 81 iterations to logL = 2690996.133907881
scarlet ran for 81 iterations to logL = 2690997.2060056473
scarlet ran for 81 iterations to logL = 2690997.8501902157
scarlet ran for 6 iterations to logL = 2688704.3980828393
30.629587013622192 31.782612241991913
/home/majoburo/Work/GWHET-paper/cubes/kn_2_4000_20191214_0000023_047_cube.fits
scarlet ran for 57 iterations to logL = 2690818.1275417693
scarlet ran for 80 iterations to logL = 2690605.3636029833
scarlet ran for 36 iterations to logL = 2690414.0831858446
scarlet ran for 45 iterations to logL = 2690404.9154357943
scarlet ran for 85 iterations to logL = 2690339.2753881854
scarlet ran for 161 iterations to logL = 2689457.474546054
scarl

In [ ]:
th_amp